Date: 8-Dec-23

Time Series forecating (using Facebook Prophet): Air quality (Bangkok)
- Learning from: Prasert Kanawattanachai (CBS)
- Youtube: https://www.youtube.com/prasertcbs
- Github: https://github.com/prasertcbs/
- Dataset: bangkok-air-quality.csv (source: https://aqicn.org/data-platform/register/)
- Facebook Prophet: https://github.com/facebook/prophet

In [1]:
# import libraries

import sys
import pandas as pd
import numpy as np
# import math
import matplotlib.pylab as plt
import seaborn as sns

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters() 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
print(f'Pythoon version : {sys.version}')
print(f'pandas version  : {pd.__version__}')
print(f'numpy version   : {np.__version__}')
print(f'seaborn version : {sns.__version__}')

pd.Timestamp.now()

Pythoon version : 3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:48:25) 
[Clang 14.0.6 ]
pandas version  : 1.5.2
numpy version   : 1.25.2
seaborn version : 0.12.2


Timestamp('2023-12-08 07:19:54.903218')

In [3]:
import warnings

warnings.filterwarnings('ignore')

In [4]:
# read data to a dataframe

url = 'https://github.com/prasertcbs/basic-dataset/raw/master/bangkok-air-quality.csv'
data = pd.read_csv(url)
data.head()

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:
data.info()

In [5]:
data = pd.read_csv(url, parse_dates = ['date'], na_values = ' ', skipinitialspace = True)
data.head()

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:
data.info()

In [ ]:
data = data.sort_values('date').reset_index(drop = True).copy() # sort date and reset index - ascending sort
data

In [ ]:
y_col = 'pm25' # forecast column
y_col

In [ ]:
# drop NA/Null referred to column pm25

data.dropna(subset = [y_col], inplace = True)

In [ ]:
data

In [ ]:
xyz

In [ ]:
# reset index

data = data.reset_index(drop = True)
data

In [ ]:
data.info()

In [ ]:
# create a function to separate year, month, day, day name (Sunday-Saturday)

def date_parts(data, date_col = 'date'):
    
    ''' 
    create new columns for year, month, day, and day name
    '''

    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['day'] = data['date'].dt.day
    data['day_name'] = data['date'].dt.day_name().astype('category')

In [ ]:
# call a function above to get new data from 'date' column

date_parts(data)

In [ ]:
data

In [ ]:
data.columns

In [ ]:
# select columns and save to new dataframe

df = data[['date', 'year', 'month', 'day', 'day_name', y_col]]
df

In [ ]:
# set date to be an index

df = df.set_index('date')
df

### Visualize data

In [ ]:
plt.figure(figsize = (16, 9))
plt.scatter(df.index, df[y_col], alpha = .5, s = 4, label = 'actual', color = '.4')
plt.ylabel('PM 2.5')
plt.title('Bangkok PM 2.5')
plt.legend();

### lightgbm regressor

In [ ]:
df.columns

In [ ]:
feature_cols = ['year', 'month', 'day', 'day_name']
feature_cols

In [ ]:
X = df[feature_cols]
X.tail()

In [ ]:
y = df[y_col]
y.tail()

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

print(f'lightgbm version: {lgb.__version__}')

In [ ]:
# split data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .1, random_state = 1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


In [ ]:
split_at = 1400

X_train, X_test, y_train, y_test = X[:split_at], X[split_at:], y[:split_at], y[split_at:]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# create a model

reg = lgb.LGBMRegressor()
reg

In [ ]:
reg.get_params() # default params

In [ ]:
# set params

params = {'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1, # set to be -1
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [ ]:
# create a model using set params

reg = lgb.LGBMRegressor(**params)
reg

In [ ]:
# train/fit a model

reg.fit(X_train, y_train)

In [ ]:
reg.__dict__

In [ ]:
# get training score

reg.score(X_train, y_train)

In [ ]:
# get testing score

reg.score(X_test, y_test)

In [ ]:
# predict y using first 5 X

reg.predict(X_test[:5])

### Visualize forecast

In [ ]:
df.head()

In [ ]:
plt.figure(figsize = (16, 9))
plt.scatter(df.index, df[y_col], alpha = .5, s = 4, label = 'actual', color = '.4') # original/actual data
plt.scatter(X_train.index, reg.predict(X_train), alpha = .5, s = 4, label = 'predicted_X_train', color = 'deepskyblue') # predicted y using X_train
plt.scatter(X_test.index, reg.predict(X_test), alpha = .5, s = 4, label = 'predicted_X_test', color = 'orange') # predicted y using X_test
plt.ylabel('PM 2.5')
plt.title('Bangkok PM 2.5')
plt.legend();

In [ ]:
df

### SHAP

In [ ]:
np.__version__

In [ ]:
# pip install numba --upgrade # could not import shap and required to upgrade numba

In [ ]:
# import a library

import shap

In [ ]:
# load JS visualization code to notebook

shap.initjs()

In [ ]:
X

In [ ]:
X.tail

In [ ]:
X.loc[['2021-03-11']]

In [ ]:
Xi = pd.DataFrame([[2016, 3, 17, 'Wednesday']])
Xi

In [ ]:
Xi = np.array([[2016, 3, 17, 6]])
Xi

In [ ]:
# explain the model's predictions using SHAP

explainer = shap.TreeExplainer(reg)
explainer

In [ ]:
# shap_values = explainer.shap_values(X.loc[['2021-03-04']])
# shap_values = explainer.shap_values(np.array([[2016,3,17,6]]))
shap_values = explainer.shap_values(X)
shap_values

In [ ]:
explainer.__dict__

In [ ]:
reg.predict(X_train).mean() # explainer.expexted_value

In [ ]:
shap_values[:3]

In [ ]:
shap_values.shape

In [ ]:
X[:5]

In [ ]:
feature_cols

In [ ]:
# shap values of each case

dshap = pd.DataFrame(shap_values, columns = feature_cols)
dshap

In [ ]:
X[:5]

In [ ]:
X.tail()

In [ ]:
np.abs(dshap).mean().sort_values(ascending = False)

In [ ]:
shap.summary_plot(shap_values, X, plot_type = 'bar')

In [ ]:
# summarize the effects of all the features

shap.summary_plot(shap_values, X)

In [ ]:
explainer.shap_values(X)

In [ ]:
# create funtions

def case_detail(case_data):

    '''
    format obj returned from shap.force_plot()
    '''

    de = pd.DataFrame(case_data.data['features'])
    fcols = []

    for i in case_data.data['features'].keys():
        fcols.append(case_data.data['featureNames'][i])

    de.columns = fcols
    
    return de

def individual_case_plot(explainer, X, case_index):

    """
    >>> individual_case_plot(explainer, X_train, 1)
    """
    shap_values = explainer.shap_values(X.iloc[[case_index]])
    g = shap.force_plot(explainer.expected_value, shap_values = shap_values, features = X.iloc[case_index, :])
    
#     shap_values = explainer.shap_values(X)
#     g=shap.force_plot(explainer.expected_value, shap_values=shap_values[case_index], features=X.iloc[case_index, :])
#     print('SHAP values')
#     dshap=pd.DataFrame(shap_values, columns=feature_cols)
#     dpos=dshap.iloc[case_index][dshap.iloc[case_index].values > 0].sort_values(ascending=False)
#     dneg=dshap.iloc[case_index][dshap.iloc[case_index].values < 0].sort_values(ascending=False)
#     print(dpos)
#     print(f'sum pos = {dpos.sum():.6f}')
#     print(dneg)
#     print(f'sum neg = {dneg.sum():.6f}')

#     print(f'sum SHAP = {dpos.sum()+dneg.sum():.6f}')
#     print(f'base      value = {explainer.expected_value:.2f}')
#     print(f'predicted value = {explainer.expected_value + dpos.sum() + dneg.sum():.2f}')
#     pprint(g.data)
    return g

In [ ]:
X[:3]

In [ ]:
X.iloc[[3]]

In [ ]:
X.iloc[[2]]

In [ ]:
# call a functin

individual_case_plot(explainer, X, 0)

In [ ]:
g = individual_case_plot(explainer, X, 0)
g

In [ ]:
g.__dict__

In [ ]:
de = case_detail(g)
de

In [ ]:
de.loc['effect', :].sum()

In [ ]:
de.loc['effect', :].sum() + g.data['baseValue']

In [ ]:
sum_effect = 0

for k, v in g.data['features'].items():

    sum_effect += v['effect']
    
sum_effect

In [ ]:
g.data['baseValue'] + sum_effect

In [ ]:
g = individual_case_plot(explainer, X, 120)
g

In [ ]:
case_detail(g)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[: 365], X[: 365])

In [ ]:
X